These brochure credit card product pdf i got from google search engine

In [1]:
!pip install langchain-community
!pip install tiktoken
!pip install faiss-cpu
!pip install PyPDF2  # Install PyPDF2 for PDF handling
!pip install camelot-py[base] # Install camelot for tabular data extraction
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[base] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.9 MB/s eta 0:00:00


In [2]:
import PyPDF2

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text

# Extract text from the available PDF files
bank_rakyat = extract_text_from_pdf('/content/bank-rakyat-credit-card.pdf')
public_bank = extract_text_from_pdf('/content/pds-credit-card-en.pdf')
hong_leong = extract_text_from_pdf('/content/hlb-product-disclosure-sheet-en.pdf')

In [3]:
import re

# Function to split text into chunks
def split_text_into_chunks(text, chunk_size=500):
    # Split text into words
    words = re.findall(r'\S+', text)
    # Create chunks
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Use the extracted text from the PDFs
texts = [bank_rakyat,public_bank, hong_leong]

# Split each text into chunks
chunks = []
for text in texts:
    chunks.extend(split_text_into_chunks(text))

# Display the first few chunks to verify
chunks[:5]

["Product Disclosure S heet – Credit Card -i Bank Kerjasama Rakyat Malaysia Berhad Version 13/Dated 25/3/2024/BKRM PRODUCT DISCLOSURE SHEET IMPORTANT : Please read this Product Disclosure Sheet before you decide to take Bank Rakyat Credit Card -i. Please ensure that you also read the general Terms and Conditions. BANK RAKYAT Bank Rakyat Credit Card -i Date : 1. What is Ban k Rakyat Credit Card -i? This is a MasterCard Credit Card -i, with a financing limit granted by us to you where if the financing limit which has been utilized but has not been fully settled before the due date will be subject to profit charges. 2. What is the Shariah concept appli cable? The financing facili ties are under the Shariah principles of Murabahah (by adopting Tawarruq practice) where based on Wa’d from customers, the Bank will purchase commodities from commodity suppliers and subsequently sell commo dities to customers based on Murabahah contract at selling price (commodity cost with profit) by def erred 

In [4]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document # Import Document class
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = '#'

# Convert chunks (strings) to Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# Insert question & answer csv file
db = FAISS.from_documents(documents, OpenAIEmbeddings()) # Use documents instead of chunks

<ipython-input-4-1a1a2b8fe128>:18: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(documents, OpenAIEmbeddings()) # Use documents instead of chunks


In [5]:
query = "Bank Rakyat minimum requirement : applicant who earns: ➢ <RM36,000 per annum .  "
docs = db.similarity_search(query)
print(docs[0].page_content)

assessment on limit eli gibility. 2. Applicants who are unable to provide pro of of income do cument, you must open Bank Rakyat Term Deposit -i Account and place a total amount of deposit which will be equivalent to the total Credit C ard-i financing limit applied. The deposit in the Te rm Deposit -i Account shall be mainta ined for as long as the Credit Card -i is active. Product Disclosure S heet – Credit Card -i Bank Kerjasama Rakyat Malaysia Berhad Version 13/Dated 25/3/2024/BKRM b) LOW annual profit rate under the Bank Rakyat Credit Card -i Tiered Pricing Structure s: Annual Profit Rat e Criteria Tier Retail 13.5% Non-Bank Rakyat Member Tier 1: Minimum payment received before or on the Payment Due Date for 9 consecutive months . 13.0% Bank Rakyat Member 17.0% Non-Bank Rakyat Member Tier 2: Minimu m payment received before or on the Payment D ue Date for 6 months or more within 9 months period. 15.5% Bank Rakyat Member 18.0% Non-Bank Rakyat Member Tier 3: Does not meet the above co

In [6]:
print(documents[1])

page_content='assessment on limit eli gibility. 2. Applicants who are unable to provide pro of of income do cument, you must open Bank Rakyat Term Deposit -i Account and place a total amount of deposit which will be equivalent to the total Credit C ard-i financing limit applied. The deposit in the Te rm Deposit -i Account shall be mainta ined for as long as the Credit Card -i is active. Product Disclosure S heet – Credit Card -i Bank Kerjasama Rakyat Malaysia Berhad Version 13/Dated 25/3/2024/BKRM b) LOW annual profit rate under the Bank Rakyat Credit Card -i Tiered Pricing Structure s: Annual Profit Rat e Criteria Tier Retail 13.5% Non-Bank Rakyat Member Tier 1: Minimum payment received before or on the Payment Due Date for 9 consecutive months . 13.0% Bank Rakyat Member 17.0% Non-Bank Rakyat Member Tier 2: Minimu m payment received before or on the Payment D ue Date for 6 months or more within 9 months period. 15.5% Bank Rakyat Member 18.0% Non-Bank Rakyat Member Tier 3: Does not mee

In [7]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chains.llm import LLMChain # Import LLMChain
from langchain.prompts import PromptTemplate # Import PromptTemplate

retriever = db.as_retriever()

# Create a custom prompt template with a reduced context window
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT) # Chain with custom prompt

<ipython-input-7-54932caa22e7>:18: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT) # Chain with custom prompt
<ipython-input-7-54932caa22e7>:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT) # Chain with custom prompt


In [8]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What are the typical benefits and rewards I should look for when selecting my first credit card?"
qa.run(query)

<ipython-input-8-477f40976d58>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


"\nSome typical benefits and rewards to look for when selecting your first credit card may include:\n\n1. Cash back or rewards program: Many credit cards offer cash back on purchases or a rewards program where you can earn points that can be redeemed for travel, merchandise, or cash.\n\n2. No annual fee: Some credit cards charge an annual fee, so it's important to look for one that does not have this fee.\n\n3. Low interest rate: If you plan on carrying a balance on your credit card, look for one with a low interest rate to avoid paying high finance charges.\n\n4. Introductory offers: Some credit cards may offer an introductory 0% APR for a certain period of time, making it a good option for large purchases or balance transfers.\n\n5. Fraud protection: Credit cards offer protection against fraudulent charges, so it's important to choose one that has strong security measures in place.\n\n6. Additional perks: Some credit cards may offer additional perks such as travel insurance, airport 


Some typical benefits and rewards to look for when selecting your first credit card may include:

1. Cash back or rewards program: Many credit cards offer cash back on purchases or a rewards program where you can earn points that can be redeemed for travel, merchandise, or cash.

2. No annual fee: Some credit cards charge an annual fee, so it's important to look for one that does not have this fee.

3. Low interest rate: If you plan on carrying a balance on your credit card, look for one with a low interest rate to avoid paying high finance charges.

4. Introductory offers: Some credit cards may offer an introductory 0% APR for a certain period of time, making it a good option for large purchases or balance transfers.

5. Fraud protection: Credit cards offer protection against fraudulent charges, so it's important to choose one that has strong security measures in place.

6. Additional perks: Some credit cards may offer additional perks such as travel insurance, airport lounge access, or discounts at certain retailers.

When selecting your first credit card, it's important to consider your spending habits and needs to find the best fit for you. It's also important to carefully read the terms and conditions and understand any potential fees or limitations before choosing a credit card

In [10]:
query = "How much monthly payment credit card"
qa.run(query)

' The minimum monthly payment for the credit card is 5% of the outstanding balance or a minimum of RM50, whichever is higher.'

In [11]:
query = "What is the maximum age for approval?"
qa.run(query)

" The maximum age for approval varies depending on the specific product and takaful coverage. For example, the maximum age for takaful coverage for the Rakyat Rebate is 64 years old on the date of the event (death/TPD). It is best to check with the specific product's terms and conditions for more information on age eligibility."

In [12]:
query = "What are the career requirements to be eligible for this credit card from different banks?"
qa.run(query)

'\nThe career requirements to be eligible for a credit card from different banks may vary. Some common requirements may include:\n\n1. Employment status: Most banks require applicants to have a stable and regular source of income, such as being employed full-time or self-employed.\n\n2. Minimum income: Banks may have a minimum income requirement for credit card applicants, which can vary from bank to bank. This is to ensure that the applicant has the financial means to repay the credit card debt.\n\n3. Credit score: A good credit score is usually necessary to be eligible for a credit card. This shows the bank that you have a responsible credit history and are likely to repay your debts on time.\n\n4. Age: Most banks require applicants to be at least 18 years old to be eligible for a credit card. Some may have a higher minimum age requirement.\n\n5. Citizenship or residency: Some banks may only offer credit cards to citizens or permanent residents of the country where the bank is locate

In [13]:
query = "How can I effectively manage my spending and avoid debt with my new credit card?"
qa.run(query)

"Some tips for effectively managing your spending and avoiding debt with your new credit card are:\n1. Set a budget and stick to it: Before using your credit card, create a budget for your monthly expenses and make sure to stay within that budget. This will help you avoid overspending and accumulating debt.\n2. Use your credit card for necessary expenses only: Avoid using your credit card for impulse purchases or non-essential items. Stick to using it for necessary expenses such as groceries, gas, and bills.\n3. Pay your balance in full each month: Make it a goal to pay off your full credit card balance each month. This will help you avoid paying interest and accumulating debt.\n4. Monitor your spending: Keep track of your credit card transactions and regularly check your credit card statement. This will help you stay aware of your spending and identify any unauthorized charges.\n5. Avoid cash advances: Cash advances come with high interest rates and fees, so try to avoid using your cr

In [14]:
query = "How does using a credit card responsibly help me build my credit score, and what should I do to maintain a good standing?"
qa.run(query)

" Using a credit card responsibly can help you build your credit score because it shows lenders that you are able to manage credit effectively. To maintain a good standing, make sure to pay your credit card bill on time and in full each month, keep your credit utilization low, and only use credit for necessary purchases. It's also important to regularly check your credit report for any errors or fraudulent activity."

In [15]:
query = "What fees and interest rates should I be aware of with my first credit card, and how can I minimize them?"
qa.run(query)

" When getting your first credit card, it's important to understand the fees and interest rates associated with it. Some fees to be aware of include an annual fee, which is charged every year for having the card, and a late payment fee if you miss a payment. Interest rates, or APR, will also be charged on any balance you carry over from month to month. To minimize these fees and rates, make sure to pay your balance in full and on time every month. You can also look for a card with no annual fee and a low interest rate. It's important to read and understand all terms and fees associated with your credit card before applying."